In [1]:
# Imports
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from web3 import Web3
from datetime import datetime, timedelta, timezone

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json'),
    'AAVE_WBP': ('0xC7be5307ba715ce89b152f3Df0658295b3dbA8E2', './WalletBalanceProvider_ABI.json'),
    'COMPOUND': ('0xA17581A9E3356d9A858b789D68B4d866e593aE94', './Compound_ABI.json')
    # fill with rest
}

# The address for the token we are trying to see the balances for, in this case the receipt token of rETH - "aethrETH"
receipt_tokens = {
'rETH': ('0xCc9EE9483f662091a1de4795249E24aC0aC2630f', 'aethrETH'),
'USDT': ('0x71fc860F7D3A592A4a98740e39dB31d25db65ae8', 'aUSDT'),
'USDC': ('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c', 'aethUSDC'),
'DAI': ('0x4C612E3B15b96Ff9A6faED838F8d07d479a8dD4c', 'aethsDAI'),
'WBTC': ('0xFC4B8ED459e00e5400be803A9BB3954234FD50e3', 'aWBTC'),
'WETH': ('0x030bA81f1c18d280636F32af80b9AAd02Cf0854e', 'aWETH')
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [2]:
import re

code = '''
POOL_ADDRESSES_PROVIDER = IPoolAddressesProvider(0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e);
POOL = IPool(0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2);
POOL_IMPL = 0x5FAab9E1adbddaD0a08734BE8a52185Fd6558E14;
POOL_CONFIGURATOR = IPoolConfigurator(0x64b761D848206f447Fe2dd461b0c635Ec39EbB27);
POOL_CONFIGURATOR_IMPL = 0xFDA7ffA872bDc906D43079EA134ebC9a511db0c2;
ORACLE = IAaveOracle(0x54586bE62E3c3580375aE3723C145253060Ca0C2);
AAVE_PROTOCOL_DATA_PROVIDER = IPoolDataProvider(0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3);
ACL_MANAGER = IACLManager(0xc2aaCf6553D20d1e9d78E365AAba8032af9c85b0);
ACL_ADMIN = 0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A;
COLLECTOR = ICollector(0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c);
DEFAULT_INCENTIVES_CONTROLLER = 0x8164Cc65827dcFe994AB23944CBC90e0aa80bFcb;
DEFAULT_A_TOKEN_IMPL_REV_1 = 0x7EfFD7b47Bfd17e52fB7559d3f924201b9DbfF3d;
DEFAULT_VARIABLE_DEBT_TOKEN_IMPL_REV_1 = 0xaC725CB59D16C81061BDeA61041a8A5e73DA9EC6;
DEFAULT_STABLE_DEBT_TOKEN_IMPL_REV_1 = 0x15C5620dfFaC7c7366EED66C20Ad222DDbB1eD57;
EMISSION_MANAGER = 0x223d844fc4B006D67c0cDbd39371A9F73f69d974;
CAPS_PLUS_RISK_STEWARD = 0x82dcCF206Ae2Ab46E2099e663F70DeE77caE7778;
FREEZING_STEWARD = 0x2eE68ACb6A1319de1b49DC139894644E424fefD6;
DEBT_SWAP_ADAPTER = 0x8761e0370f94f68Db8EaA731f4fC581f6AD0Bd68;
DELEGATION_AWARE_A_TOKEN_IMPL_REV_1 = 0x21714092D90c7265F52fdfDae068EC11a23C6248;
CONFIG_ENGINE = 0xA3e44d830440dF5098520F62Ebec285B1198c51E;
POOL_ADDRESSES_PROVIDER_REGISTRY = 0xbaA999AC55EAce41CcAE355c77809e68Bb345170;
RATES_FACTORY = 0xcC47c4Fe1F7f29ff31A8b62197023aC8553C7896;
REPAY_WITH_COLLATERAL_ADAPTER = 0x02e7B8511831B1b02d9018215a0f8f500Ea5c6B3;
STATIC_A_TOKEN_FACTORY = 0x411D79b8cC43384FDE66CaBf9b6a17180c842511;
SWAP_COLLATERAL_ADAPTER = 0xADC0A53095A0af87F3aa29FE0715B5c28016364e;
UI_GHO_DATA_PROVIDER = 0x379c1EDD1A41218bdbFf960a9d5AD2818Bf61aE8;
UI_INCENTIVE_DATA_PROVIDER = 0x162A7AC02f547ad796CA549f757e2b8d1D9b10a6;
UI_POOL_DATA_PROVIDER = 0x91c0eA31b49B69Ea18607702c5d9aC360bf3dE7d;
WALLET_BALANCE_PROVIDER = 0xC7be5307ba715ce89b152f3Df0658295b3dbA8E2;
WETH_GATEWAY = 0x893411580e590D62dDBca8a703d61Cc4A8c7b2b9;
WITHDRAW_SWAP_ADAPTER = 0x78F8Bd884C3D738B74B420540659c82f392820e0;
SAVINGS_DAI_TOKEN_WRAPPER = 0xE28E2c8d240dd5eBd0adcab86fbD79df7a052034;
WRAPPED_TOKEN_GATEWAY = 0xD322A49006FC828F9B5B37Ab215F99B4E5caB19C;
}
'''

# Split the string into lines
lines = code.split('\n')

known_addresses = {}
for line in lines:
    # Splitting the line into variable name and the rest
    parts = line.split('=')
    if len(parts) == 2:
        variable_name = parts[0].strip()
        # Extracting the Ethereum address
        ethereum_address = re.search(r'0x[a-fA-F0-9]{40}', parts[1])
        if ethereum_address:
            known_addresses[ethereum_address.group()] = variable_name
            
known_addresses

{'0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e': 'POOL_ADDRESSES_PROVIDER',
 '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2': 'POOL',
 '0x5FAab9E1adbddaD0a08734BE8a52185Fd6558E14': 'POOL_IMPL',
 '0x64b761D848206f447Fe2dd461b0c635Ec39EbB27': 'POOL_CONFIGURATOR',
 '0xFDA7ffA872bDc906D43079EA134ebC9a511db0c2': 'POOL_CONFIGURATOR_IMPL',
 '0x54586bE62E3c3580375aE3723C145253060Ca0C2': 'ORACLE',
 '0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3': 'AAVE_PROTOCOL_DATA_PROVIDER',
 '0xc2aaCf6553D20d1e9d78E365AAba8032af9c85b0': 'ACL_MANAGER',
 '0x5300A1a15135EA4dc7aD5a167152C01EFc9b192A': 'ACL_ADMIN',
 '0x464C71f6c2F760DdA6093dCB91C24c39e5d6e18c': 'COLLECTOR',
 '0x8164Cc65827dcFe994AB23944CBC90e0aa80bFcb': 'DEFAULT_INCENTIVES_CONTROLLER',
 '0x7EfFD7b47Bfd17e52fB7559d3f924201b9DbfF3d': 'DEFAULT_A_TOKEN_IMPL_REV_1',
 '0xaC725CB59D16C81061BDeA61041a8A5e73DA9EC6': 'DEFAULT_VARIABLE_DEBT_TOKEN_IMPL_REV_1',
 '0x15C5620dfFaC7c7366EED66C20Ad222DDbB1eD57': 'DEFAULT_STABLE_DEBT_TOKEN_IMPL_REV_1',
 '0x223d844fc4B006D6

In [3]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')



# Lists transactions of supplying rETH to specified markets
def supply_transactions(market, timeframe = 30, token = 'rETH'):
    """
    Returns all "Supply" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: supply_transactions(AAVE, 90, rETH) returns all supply transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique supply transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    supply_list = []
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Make sure the requested market is supported
    if market in contract_address_abi_map.keys():
        # Set up contract address and ABI for each market
        contract_address, abi_filepath = contract_address_abi_map[market]

        # Web3 setup
        with open(abi_filepath) as f:
            abi = json.load(f)

        smart_contract = w3.eth.contract(address=contract_address, abi=abi)

        # Here we will need some sort of if/else or switching logic to process unique calls for each market
        # In the meantime, we simply check for the only supported market, 'AAVE'
        if market == 'AAVE':
            supplies = smart_contract.events.Supply().get_logs(fromBlock=st_block,
                                                       toBlock='latest',
                                                       argument_filters={'reserve': token_address})
            # Consider simplifying supply_list.. don't know what information future markets will return, 
            # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
            supply_list = [
                {
                'Reserve': event['args']['reserve'],
                'User (Wallet Address)': event['args']['user'],
                'Amount (Wei)': event['args']['amount'],
                'Amount (rETH)': event['args']['amount']/10**18,
                'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                'LogIndex': event['logIndex'],
                'TransactionIndex': event['transactionIndex'],
                'TransactionHash': event['transactionHash'].hex(),
                'BlockHash': event['blockHash'].hex(),
                'BlockNumber': event['blockNumber']
                }
                for event in supplies
            ]
        elif market == 'COMPOUND':
            supplies = smart_contract.events.SupplyCollateral().get_logs(fromBlock=st_block,
                                                toBlock='latest',
                                                argument_filters={'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393'})
            supply_list = [
                {
                'Reserve': event['args']['asset'],
                'User (Wallet Address)': event['args']['from'],
                'Amount (Wei)': event['args']['amount'],
                'Amount (rETH)': event['args']['amount']/10**18,
                'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                'LogIndex': event['logIndex'],
                'TransactionIndex': event['transactionIndex'],
                'TransactionHash': event['transactionHash'].hex(),
                'BlockHash': event['blockHash'].hex(),
                'BlockNumber': event['blockNumber']
                }
                for event in supplies
            ]
        else:
            print("FUTURE MARKETS HERE")
            return

    # The market wasn't in our dictionary of supported markets
    else:
        print("UNSUPPORTED MARKET ENTERED")

    # Push withdrawls into dataframe
    recent_supplies = pd.DataFrame(supply_list)
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_supplies

In [13]:
# Function was made due to an error in processing "WETH". Due to the coingecko returning a slighly different contract address value,
# ... A solution had to be made that assumed no knowledge of the dict key value (Couldn't use "WETH_price_data[receipt_tokens['WETH'][0].lower()]", because the key 
# ... in "WETH_price_data" doesn't match our entry in "receipt_tokens['WETH'][0]"
def process_inner_dict(outer_dict):
    for inner_dict in outer_dict.values():
        if 'usd' in inner_dict:
            usd_value = inner_dict['usd']
            return usd_value
        else:
            print("INVALID DICTIONARY - No 'usd' key found in inner dictionary")

In [14]:
import requests
def fetch_price(asset):
    
    # Verify asset parameter
    if asset not in receipt_tokens.keys():
        print("INVALID ASSET")
        return
    
    price_response = requests.get("https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={}&vs_currencies=usd".format(receipt_tokens[asset][0]))
    price_data = price_response.json()
    
    asset_price = process_inner_dict(price_data)
    # Another way that is more 'pythonic', but assumes knowledge of the key -- In the case that coingecko doesn't return exactly the same address as what we supplied, this method won't work
    # asset_price = price_data[receipt_tokens[asset][0].lower()]['usd']
    return asset_price

In [10]:
def leveraged_users():
    # List used for tracking significant wallets
    positive_balances = []
    USD_balances = []
    
    # Retrieve our initial list of possible wallets using "supply_transactions" with the default timeframe for all markets. 
    # Additional parameter for timeframe could be added, defaults to 30 days [Ex. supply_transactions('AAVE', 90) for 90 days]
    recent_suppliers_aave = supply_transactions('AAVE')
    recent_suppliers_comp = supply_transactions('COMPOUND')
    recent_suppliers = pd.concat([recent_suppliers_aave, recent_suppliers_comp], ignore_index=True)
    
    # Isolate user wallets from our list of recent supply transactions
    unique_users = list(set(recent_suppliers['User (Wallet Address)']))
    
    # Remove known/benign wallets
    for wallet in unique_users:
        if wallet in known_addresses.keys():
            unique_users.remove(wallet)
            
    # Lookup token address
    token_address = token_address_map['rETH']
    
    # LOAD CONTRACTS AND ABI FILES FOR EACH MARKET SUPPORTED
    aave_address, aave_abi = contract_address_abi_map['AAVE_WBP']
    compound_address, compound_abi = contract_address_abi_map['COMPOUND']
    
    # Web3 setup
    with open(aave_abi) as file1, open(compound_abi) as file2:
        WBP_abi = json.load(file1)
        comp_abi = json.load(file2)
    
    # Load contracts
    aave_contract = w3.eth.contract(address=aave_address, abi=WBP_abi)
    comp_contract = w3.eth.contract(address=compound_address, abi=comp_abi)
    
    # Iterate through each wallet address and get the balance for rETH token and critical assets
    for wallet_address in unique_users:
        # Output from balanceOf is in wei, convert to true value using "/10**18"
        # balanceOf function must be called for each market and then added up
        rETH_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['rETH'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['rETH'][0]).call())/ 10**18))
        USDT_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['USDT'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['USDT'][0]).call())/ 10**18))
        USDC_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['USDC'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['USDC'][0]).call())/ 10**18))
        DAI_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['DAI'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['DAI'][0]).call())/ 10**18))
        WBTC_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['WBTC'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['WBTC'][0]).call())/ 10**18))
        WETH_receipt_balance = (((aave_contract.functions.balanceOf(wallet_address, receipt_tokens['WETH'][0]).call())/ 10**18) + 
                                ((comp_contract.functions.balanceOf(wallet_address, receipt_tokens['WETH'][0]).call())/ 10**18))

        # If the user currently has a zero balance of the rETH receipt token --> no way they can be leveraged 
        if rETH_receipt_balance > 0:
            net_asset_borrow = USDC_receipt_balance + DAI_receipt_balance + WBTC_receipt_balance + WETH_receipt_balance
            formatted_log = {
                    'User (Wallet Address)': wallet_address,
                    'rETH Balance': rETH_receipt_balance,
                    'USDT Balance': USDT_receipt_balance,
                    'USDC Balance': USDC_receipt_balance,
                    'DAI Balance': DAI_receipt_balance,
                    'WBTC Balance': WBTC_receipt_balance,
                    'WETH Balance': WETH_receipt_balance,
                    }
            positive_balances.append(formatted_log)
    
    # Save all of our positive balance wallets into a new dataframe
    sus_wallets = pd.DataFrame(positive_balances)
    
    # Retrieve real-time price information from assets for leverage calculation
    rETH_price = fetch_price('rETH')
    USDT_price = fetch_price('USDT')
    USDC_price = fetch_price('USDC')
    DAI_price = fetch_price('DAI')
    WBTC_price = fetch_price('WBTC')
    WETH_price = fetch_price('WETH')
    
    # Calculate balances for each asset in each wallet based on USD price
    for index, user in enumerate(positive_balances):
        rETH_USD_balance = positive_balances[index]['rETH Balance'] * rETH_price
        USDT_USD_balance = positive_balances[index]['USDT Balance'] * USDT_price
        USDC_USD_balance = positive_balances[index]['USDC Balance'] * USDC_price
        DAI_USD_balance = positive_balances[index]['DAI Balance'] * DAI_price
        WBTC_USD_balance = positive_balances[index]['WBTC Balance'] * WBTC_price
        WETH_USD_balance = positive_balances[index]['WETH Balance'] * WETH_price

        net_asset_borrow = USDT_USD_balance + USDC_USD_balance + DAI_USD_balance + WBTC_USD_balance + WETH_USD_balance
        formatted_log = {
                'User (Wallet Address)': wallet_address,
                'rETH Balance': rETH_USD_balance,
                'USDT Balance': USDT_USD_balance,
                'USDC Balance': USDC_USD_balance,
                'DAI Balance': DAI_USD_balance,
                'WBTC Balance': WBTC_USD_balance,
                'WETH Balance': WETH_USD_balance,
                'Net Asset Borrow': net_asset_borrow
                }
        USD_balances.append(formatted_log)
        
    sus_wallets_usd = pd.DataFrame(USD_balances)

In [15]:
# TO DO
# At this point, we are able to identify leveraged users based on the net total of assets borrowed (Balance of ll assets besides rETH added up)
    # and the notional amount of rETH deposited (rETH Balance)
# From this point onward, this information can be catalogued daily to be used for historical lookback. In this way, today would be day 1, and the furthest we would be able to "look back"
# Conducting historical lookback for previous timeperiods would require a "balanceOf" function which, instead of returning the current balance, returns the balance of a given wallet 
    # at some point (timestamp) in the past. This may be possible, but hasn't yet been explored. 
# Note that we also believed it should be possible to simply acquire the current holders of a particular token using some web3 command like "holders", or some mapping ("balances")
    # This method is being explored and should replace the need for our borrow_transactions function, which currently makes the incorrect assumption that anyone who is leveraged
    # must have submitted a borrow transaction within the last x number of days (30 days in the present implementation).